<a href="https://www.kaggle.com/code/roninnapoletano/mgl8707-enrondataset?scriptVersionId=139282509" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import time
import email

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, auc, roc_curve, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# **Jeu de données**

Ce jeu est tiré de du EnronDataset (https://www.cs.cmu.edu/~enron/)

# **Contexte**
Dans le contexte actuel de communication, les e-mails jouent un rôle essentiel dans les échanges professionnels et personnels. Malheureusement, les e-mails de spam représentent une menace.

C'est dans ce contexte que le projet de détection d'e-mails de spam ou non-spam prend  son sens. L'objectif est de développer un système de détection efficace qui peut automatiquement identifier et filtrer les e-mails de spam, tout en minimisant les faux positifs.



In [2]:
dataset = pd.read_csv("/kaggle/input/enrondataset/emails.csv")

# **1. Exploration des données**

Exploration du jeu de données
* Quels sont les types de données que nous avons ?
* Certaines données sont-elles manquantes ?
* Quelle est la répartition des données ?
* Comment les données sont elles représentées ?


In [3]:
dataset.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [4]:
dataset.head(n=20)

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...
7,allen-p/_sent_mail/1004.,Message-ID: <17189699.1075863688308.JavaMail.e...
8,allen-p/_sent_mail/101.,Message-ID: <20641191.1075855687472.JavaMail.e...
9,allen-p/_sent_mail/102.,Message-ID: <30795301.1075855687494.JavaMail.e...


In [5]:
# Afficher le nombre de lignes et de colonnes dans ce dataset
print(f'Il y a {dataset.shape[0]} lignes et {dataset.shape[1]} colonnes dans ce dataset')

# Vérifier s'il y a des données manquantes
print(f'A-t-on des données manquantes ? {dataset.isna().any()}\n\n')

# Afficher combien d'entrées ont des données manquantes
print(f'Combien d\'entrées ont des données manquantes ? {len(dataset[dataset.isnull().any(axis=1)])}')

# Afficher combien d'entrées ont des données manquantes par colonne
print(f'Combien d\'entrées ont des données manquantes par colonne ? {dataset.isnull().sum()}\n\n')

# Afficher les types de chaque variable dans le dataset
print(f'Types des variables : {dataset.info()}')

Il y a 517401 lignes et 2 colonnes dans ce dataset
A-t-on des données manquantes ? file       False
message    False
dtype: bool


Combien d'entrées ont des données manquantes ? 0
Combien d'entrées ont des données manquantes par colonne ? file       0
message    0
dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB
Types des variables : None


In [6]:
dataset.columns.values

array(['file', 'message'], dtype=object)

**Visualisation d'un email**

In [7]:
print(dataset.loc[5]['message'])

Message-ID: <30965995.1075863688265.JavaMail.evans@thyme>
Date: Thu, 31 Aug 2000 04:17:00 -0700 (PDT)
From: phillip.allen@enron.com
To: greg.piper@enron.com
Subject: Re: Hello
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Greg Piper
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

Greg,

 How about either next Tuesday or Thursday?

Phillip


In [8]:
dataset.describe()

,file,message
count,517401,517401
unique,517401,517401
top,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
freq,1,1


# **2. Transforamtions des Emails**

* Pouvons transformer extraire des meta données ?
* Des données autres que le mail sont présentes ?
* Existe-t-il des standards d'en têtes ?
* Pouvons nous créer d'autres colonnes ?

D'après https://datatracker.ietf.org/doc/html/rfc5322 et https://datatracker.ietf.org/doc/html/rfc4021 . Nous sommes en mesure d'extraire des en-têtes de mail.

In [9]:
message_str = dataset.loc[5]['message']
email_message = email.message_from_string(message_str)


email_message.items()

[('Message-ID', '<30965995.1075863688265.JavaMail.evans@thyme>'),
 ('Date', 'Thu, 31 Aug 2000 04:17:00 -0700 (PDT)'),
 ('From', 'phillip.allen@enron.com'),
 ('To', 'greg.piper@enron.com'),
 ('Subject', 'Re: Hello'),
 ('Mime-Version', '1.0'),
 ('Content-Type', 'text/plain; charset=us-ascii'),
 ('Content-Transfer-Encoding', '7bit'),
 ('X-From', 'Phillip K Allen'),
 ('X-To', 'Greg Piper'),
 ('X-cc', ''),
 ('X-bcc', ''),
 ('X-Folder', "\\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail"),
 ('X-Origin', 'Allen-P'),
 ('X-FileName', 'pallen.nsf')]

In [10]:
def extract_field(field, messages):
    fields = []
    for message in messages:
        extract_email = email.message_from_string(message)
        fields.append(extract_email.get(field))
    return fields

def extract_body(messages):
    fields = []
    for message in messages:
        extract_email = email.message_from_string(message)
        fields.append(extract_email.get_payload())
    return fields

In [11]:
dataset['Message-ID'] = extract_field("Message-ID", dataset['message'])
dataset['Date'] = extract_field("Date", dataset['message'])
dataset['Subject'] = extract_field("Subject", dataset['message'])
dataset['X-Folder'] = extract_field("X-Folder", dataset['message'])
dataset['X-From'] = extract_field("X-From", dataset['message'])
dataset['X-To'] = extract_field("X-To", dataset['message'])

dataset['Body'] =  extract_body(dataset['message'])
dataset.head(3)



,file,message,Message-ID,Date,Subject,X-Folder,X-From,X-To,Body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!


# **3. Analyse plus approfondie des Emails**

Exploration de nos nouvelles features
* Quels sont les types de données que nous avons ?
* Certaines données sont-elles manquantes ?
* Quelle est la répartition des données ?
* Comment les données sont elles représentées ?


In [12]:
# Afficher le nombre de lignes et de colonnes dans ce dataset
print(f'Il y a {dataset.shape[0]} lignes et {dataset.shape[1]} colonnes dans ce dataset')

# Vérifier s'il y a des données manquantes
print(f'A-t-on des données manquantes ? {dataset.isna().any()}\n\n')

# Afficher combien d'entrées ont des données manquantes
print(f'Combien d\'entrées ont des données manquantes ? {len(dataset[dataset.isnull().any(axis=1)])}')

# Afficher combien d'entrées ont des données manquantes par colonne
print(f'Combien d\'entrées ont des données manquantes par colonne ? {dataset.isnull().sum()}\n\n')

# Afficher les types de chaque variable dans le dataset
print(f'Types des variables : {dataset.info()}')

Il y a 517401 lignes et 9 colonnes dans ce dataset
A-t-on des données manquantes ? file          False
message       False
Message-ID    False
Date          False
Subject       False
X-Folder       True
X-From         True
X-To           True
Body          False
dtype: bool


Combien d'entrées ont des données manquantes ? 29
Combien d'entrées ont des données manquantes par colonne ? file           0
message        0
Message-ID     0
Date           0
Subject        0
X-Folder      29
X-From        29
X-To          29
Body           0
dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   file        517401 non-null  object
 1   message     517401 non-null  object
 2   Message-ID  517401 non-null  object
 3   Date        517401 non-null  object
 4   Subject     517401 non-null  object
 5   X-Folder    517372 non-null  object
 6   X-F